In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pymoo
from pymoo.core.problem import ElementwiseProblem, Problem

Przykładowe zadanie optymalizacyjne mieliśmy już podczas zajęć związanych z sieciami neuronowymi. Wtedy był to problem optymalizacji jednokryterialnej. Niestety często rzeczywiste problemy należy oceniać na więcej niż jednym kryterium i wówczas problem staje się bardziej złożonych. Sporo rozwiązań jest nieporównywalnych - nie możemy jednoznacznie wskazać które z nich jest lepsze. Wynikiem takiej optymalizacji nie będzie jedno rozwiązanie optymalne tylko zbiór rozwiązań pareto-optymalnych. Dla przykładu jeśli mamy wybrać dostawcę usług internetowych i oceniamy pod kątem prędkości i ceny to mając poniższą tabelkę:

In [2]:
df = pd.DataFrame({"cena": [30,35,40,40,45,50,70,200,50], "prędkość":[100,200,150,300,450,550,220,300,340]})
display(df)

,cena,prędkość
0,30,100
1,35,200
2,40,150
3,40,300
4,45,450
5,50,550
6,70,220
7,200,300
8,50,340


Następujące alternatywy są niezdominowane i stanowią front pareto

In [3]:
dominated = np.less_equal.outer(df.cena.values, df.cena.values) & np.greater_equal.outer(df['prędkość'].values,df['prędkość'].values) & \
(np.less.outer(df.cena.values, df.cena.values) | np.greater.outer(df['prędkość'].values,df['prędkość'].values))
df[dominated.sum(0)==0]

,cena,prędkość
0,30,100
1,35,200
3,40,300
4,45,450
5,50,550


Przykładowo opcja nr 2 jest zdominowana przez wariant 1, który przy niższej cenie oferuje lepszą prędkość. Jednak dla powyższych 5 alternatyw nie można wskazać zdominowanej

Do zadań optymalizacji wielokryterialnej bardzo dobrze nadają się algorytmy genetyczne. W swojej naturze działają one na zróżnicowanym zbiorze rozwiązań, z którego można wyciągnąć front pareto. Istnieją różne warianty algorytmów genetycznych dla optymalizacji wielokryterialnej, w których przekazujemy dodatkową informację preferencyjną, aby być w stanie porównywać warianty i skupić się na konkretnym obszarze frontu pareto, lub takie, które wymuszają, aby rozwiązania były równomiernie rozdystrybuowane po całym froncie.

Jednym z pakietów oferujących genetyczną optymalizację wielokryterialną jest pymoo. W ramach niego można korzystać z już gotowych poszczególnych etapów optymalizacji genetycznej jak i dopasować je poprzez implementację wybranych kroków.

In [4]:
from pymoo.operators.crossover.hux import HUX
from pymoo.operators.mutation.bitflip import BitflipMutation
from pymoo.operators.sampling.rnd import BinaryRandomSampling
from pymoo.optimize import minimize
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.problems.single.knapsack import create_random_knapsack_problem

problem = create_random_knapsack_problem(30)


algorithm = GA(pop_size=200,
               sampling=BinaryRandomSampling(),
               crossover=HUX(),
               mutation=BitflipMutation(),
               eliminate_duplicates=True)


res = minimize(problem,
               algorithm,
               termination=('n_gen', 20),
               verbose=True)

n_gen  |  n_eval  |     cv_min    |     cv_avg    |     f_avg     |     f_min    
     1 |      200 |  1.300000E+02 |  4.971950E+02 |             - |             -
     2 |      400 |  5.700000E+01 |  3.461950E+02 |             - |             -
     3 |      600 |  5.700000E+01 |  2.462000E+02 |             - |             -
     4 |      800 |  0.000000E+00 |  1.654450E+02 | -3.538000E+02 | -4.370000E+02
     5 |     1000 |  0.000000E+00 |  9.591000E+01 | -3.341667E+02 | -4.370000E+02
     6 |     1200 |  0.000000E+00 |  4.528000E+01 | -3.032000E+02 | -4.530000E+02
     7 |     1400 |  0.000000E+00 |  1.091500E+01 | -2.972818E+02 | -5.490000E+02
     8 |     1600 |  0.000000E+00 |  0.000000E+00 | -3.028700E+02 | -5.490000E+02
     9 |     1800 |  0.000000E+00 |  0.000000E+00 | -3.685000E+02 | -5.890000E+02
    10 |     2000 |  0.000000E+00 |  0.000000E+00 | -4.137900E+02 | -6.770000E+02
    11 |     2200 |  0.000000E+00 |  0.000000E+00 | -4.603100E+02 | -6.770000E+02
    12 |     240

Można skorzystać z gotowych problemów i gotowych rozwiązań, ale raczej nie przyczynia się to do lepszego zrozumienia dlatego najpierw zdefiniujmy problem plecakowy

In [5]:
class KnapsackProblem(ElementwiseProblem):
    def __init__(self, df, C):
        super().__init__(n_var=len(df), n_obj=1, n_ieq_constr=0)
        self.df = df
        self.C = C

    def _evaluate(self, x, out, *args, **kwargs):
        s = self.df['cap'][x].sum()
        if s > self.C:
            out["F"] = 1
        else:
            out["F"] = -self.df['val'][x].sum()
        

In [6]:
np.random.seed(42)
df = pd.DataFrame({'val': np.random.rand(50), 'cap': np.random.randint(1,10,50)})
df

,val,cap
0,0.374540,2
1,0.950714,2
2,0.731994,4
3,0.598658,8
4,0.156019,7
5,0.155995,9
6,0.058084,8
7,0.866176,5
8,0.601115,2
9,0.708073,5


In [7]:
problem = KnapsackProblem(df, 50)
algorithm = GA(pop_size=500,
               sampling=BinaryRandomSampling(),
               crossover=HUX(),
               mutation=BitflipMutation(),
               eliminate_duplicates=True)


res = minimize(problem,
               algorithm,
               termination=('n_gen', 100),
               verbose=True,
               seed=0)

n_gen  |  n_eval  |     f_avg     |     f_min    
     1 |      500 |  1.0000000000 |  1.0000000000
     2 |     1000 |  1.0000000000 |  1.0000000000
     3 |     1500 |  1.0000000000 |  1.0000000000
     4 |     2000 |  1.0000000000 |  1.0000000000
     5 |     2500 |  1.0000000000 |  1.0000000000
     6 |     3000 |  1.0000000000 |  1.0000000000
     7 |     3500 |  1.0000000000 |  1.0000000000
     8 |     4000 |  1.0000000000 |  1.0000000000
     9 |     4500 |  1.0000000000 |  1.0000000000
    10 |     5000 |  1.0000000000 |  1.0000000000
    11 |     5500 |  1.0000000000 |  1.0000000000
    12 |     6000 |  1.0000000000 |  1.0000000000
    13 |     6500 |  1.0000000000 |  1.0000000000
    14 |     7000 |  1.0000000000 |  1.0000000000
    15 |     7500 |  1.0000000000 |  1.0000000000
    16 |     8000 |  1.0000000000 |  1.0000000000
    17 |     8500 |  1.0000000000 |  1.0000000000
    18 |     9000 |  1.0000000000 |  1.0000000000
    19 |     9500 |  1.0000000000 |  1.0000000000


In [8]:
res.X

array([ True,  True,  True, False, False, False, False,  True,  True,
        True, False, False, False,  True, False, False, False, False,
        True, False, False, False,  True, False,  True,  True,  True,
        True, False, False, False, False, False,  True,  True, False,
       False, False,  True, False, False, False, False,  True, False,
        True, False,  True, False, False])

In [9]:
df[res.X].sum()

val    11.814856
cap    50.000000
dtype: float64

#### Zad
Nasza funkcja oceny ocenia jedno rozwiązanie na raz gdyż używamy ElementwiseProblem. Możemy jednak ocenić całe pokolenie na raz. Będzie to znacznie szybsze. Wykorzystaj funkcjonalność numpy i pandasa

In [1]:
class KnapsackProblem(Problem):
    def __init__(self, df, C):
        super().__init__(n_var=len(df), n_obj=1, n_ieq_constr=0)
        self.df = df
        self.C = C

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = 0 #TODO
        

NameError: name 'Problem' is not defined

In [11]:
problem = KnapsackProblem(df, 50)
algorithm = GA(pop_size=500,
               sampling=BinaryRandomSampling(),
               crossover=HUX(),
               mutation=BitflipMutation(),
               eliminate_duplicates=True)


res = minimize(problem,
               algorithm,
               termination=('n_gen', 100),
               verbose=True,
               seed=0)

n_gen  |  n_eval  |     f_avg     |     f_min    
     1 |      500 |  1.0000000000 |  1.0000000000
     2 |     1000 |  1.0000000000 |  1.0000000000
     3 |     1500 |  1.0000000000 |  1.0000000000
     4 |     2000 |  1.0000000000 |  1.0000000000
     5 |     2500 |  1.0000000000 |  1.0000000000
     6 |     3000 |  1.0000000000 |  1.0000000000
     7 |     3500 |  1.0000000000 |  1.0000000000
     8 |     4000 |  1.0000000000 |  1.0000000000
     9 |     4500 |  1.0000000000 |  1.0000000000
    10 |     5000 |  1.0000000000 |  1.0000000000
    11 |     5500 |  1.0000000000 |  1.0000000000
    12 |     6000 |  1.0000000000 |  1.0000000000
    13 |     6500 |  1.0000000000 |  1.0000000000
    14 |     7000 |  1.0000000000 |  1.0000000000
    15 |     7500 |  1.0000000000 |  1.0000000000
    16 |     8000 |  1.0000000000 |  1.0000000000
    17 |     8500 |  1.0000000000 |  1.0000000000
    18 |     9000 |  1.0000000000 |  1.0000000000
    19 |     9500 |  1.0000000000 |  1.0000000000


In [12]:
df[res.X].sum()

val    11.814856
cap    50.000000
dtype: float64

Trochę to trwało zanim zaczęły się generować akceptowalne rozwiązania - z capacity nie większym niż pojemność naszego plecaka. Postaramy się teraz zaingerować w poszczególne elementy algorytmu, aby szybciej uzyskać lepsze wyniki

Nasze ograniczenie jest na tyle proste, że można je łatwo wyrazić w formie nierówności, które są natywnie wspierane przez pymoo. Musimy w tym celu zmodyfikować definicję problemu

In [13]:
class KnapsackProblemConstrained(ElementwiseProblem):
    def __init__(self, df, C):
        super().__init__(n_var=len(df), n_obj=1, n_ieq_constr=1)
        self.df = df
        self.C = C

    def _evaluate(self, x, out, *args, **kwargs):
        s = self.df['cap'][x].sum()
        if s > self.C:
            out["F"] = 1
        else:
            out["F"] = -self.df['val'][x].sum()
        out["G"] = s - self.C
        

In [14]:
problemC = KnapsackProblemConstrained(df, 50)
algorithm = GA(pop_size=500,
               sampling=BinaryRandomSampling(),
               crossover=HUX(),
               mutation=BitflipMutation(),
               eliminate_duplicates=True)


res = minimize(problemC,
               algorithm,
               termination=('n_gen', 200),
               verbose=True,
               seed=0)

n_gen  |  n_eval  |     cv_min    |     cv_avg    |     f_avg     |     f_min    
     1 |      500 |  2.800000E+01 |  8.511200E+01 |             - |             -
     2 |     1000 |  2.200000E+01 |  6.201800E+01 |             - |             -
     3 |     1500 |  6.0000000000 |  4.589200E+01 |             - |             -
     4 |     2000 |  0.000000E+00 |  3.287000E+01 | -5.898669E+00 | -6.924837E+00
     5 |     2500 |  0.000000E+00 |  2.115200E+01 | -5.431609E+00 | -6.924837E+00
     6 |     3000 |  0.000000E+00 |  1.122400E+01 | -5.549265E+00 | -9.020056E+00
     7 |     3500 |  0.000000E+00 |  3.3800000000 | -5.455921E+00 | -9.020056E+00
     8 |     4000 |  0.000000E+00 |  0.0300000000 | -5.440005E+00 | -9.020056E+00
     9 |     4500 |  0.000000E+00 |  0.000000E+00 | -6.456785E+00 | -9.075306E+00
    10 |     5000 |  0.000000E+00 |  0.000000E+00 | -7.189458E+00 | -1.005429E+01
    11 |     5500 |  0.000000E+00 |  0.000000E+00 | -7.766529E+00 | -1.005830E+01
    12 |     600

    99 |    49500 |  0.000000E+00 |  0.000000E+00 | -1.155806E+01 | -1.188665E+01
   100 |    50000 |  0.000000E+00 |  0.000000E+00 | -1.155810E+01 | -1.188665E+01
   101 |    50500 |  0.000000E+00 |  0.000000E+00 | -1.155840E+01 | -1.188665E+01
   102 |    51000 |  0.000000E+00 |  0.000000E+00 | -1.155840E+01 | -1.188665E+01
   103 |    51500 |  0.000000E+00 |  0.000000E+00 | -1.155840E+01 | -1.188665E+01
   104 |    52000 |  0.000000E+00 |  0.000000E+00 | -1.155840E+01 | -1.188665E+01
   105 |    52500 |  0.000000E+00 |  0.000000E+00 | -1.155840E+01 | -1.188665E+01
   106 |    53000 |  0.000000E+00 |  0.000000E+00 | -1.155887E+01 | -1.188665E+01
   107 |    53500 |  0.000000E+00 |  0.000000E+00 | -1.155908E+01 | -1.188665E+01
   108 |    54000 |  0.000000E+00 |  0.000000E+00 | -1.155937E+01 | -1.188665E+01
   109 |    54500 |  0.000000E+00 |  0.000000E+00 | -1.155982E+01 | -1.188665E+01
   110 |    55000 |  0.000000E+00 |  0.000000E+00 | -1.156005E+01 | -1.188665E+01
   111 |    5550

In [15]:
df[res.X].sum()

val    11.886649
cap    50.000000
dtype: float64

Często mamy jednak do czynienia z bardziej złożonymi ograniczeniami, których nie idzie wyrazić w tak prosty sposób. Warto wtedy skorzystać z operatora naprawy, który jak sama nazwa wskazuje będzie naprawiał niepoprawne rozwiązania 

#### Zad
Zaimplementuj taki operator, który zadba o to, żeby nie przekraczaj pojemności plecaka a jednocześnie nie będzie się zbyt mocno różnił od pierwotnego rozwiązania. Zwróć uwagę, że tutaj rozważamy całe pokolenie na raz w zmiennej Z

In [16]:
from pymoo.core.repair import Repair


class ConsiderMaximumWeightRepair(Repair):

    def _do(self, problem, Z, **kwargs):
        maxCapacity = problem.C
        weights = (Z * problem.df['cap'].values).sum(axis=1)

        for i in range(len(Z)):
            pass # now repair each indvidiual i
        return Z

In [17]:
algorithm = GA(pop_size=200,
               sampling=BinaryRandomSampling(),
               crossover=HUX(),
               mutation=BitflipMutation(),
               repair=ConsiderMaximumWeightRepair(),
               eliminate_duplicates=True)


res = minimize(problem,
               algorithm,
               termination=('n_gen', 100),
               verbose=True,
               seed=0)

n_gen  |  n_eval  |     f_avg     |     f_min    
     1 |      200 | -4.090358E+00 | -7.395390E+00
     2 |      400 | -5.128944E+00 | -7.395390E+00
     3 |      600 | -5.829741E+00 | -7.492407E+00
     4 |      800 | -6.373150E+00 | -8.961713E+00
     5 |     1000 | -6.927858E+00 | -9.397898E+00
     6 |     1200 | -7.442694E+00 | -9.397898E+00
     7 |     1400 | -7.917918E+00 | -9.397898E+00
     8 |     1600 | -8.375117E+00 | -9.881085E+00
     9 |     1800 | -8.762950E+00 | -1.012279E+01
    10 |     2000 | -9.045815E+00 | -1.054795E+01
    11 |     2200 | -9.317168E+00 | -1.076790E+01
    12 |     2400 | -9.554151E+00 | -1.108298E+01
    13 |     2600 | -9.830622E+00 | -1.119624E+01
    14 |     2800 | -1.003974E+01 | -1.130281E+01
    15 |     3000 | -1.025792E+01 | -1.130281E+01
    16 |     3200 | -1.044907E+01 | -1.130281E+01
    17 |     3400 | -1.059080E+01 | -1.153168E+01
    18 |     3600 | -1.069460E+01 | -1.153168E+01
    19 |     3800 | -1.077624E+01 | -1.153168E+01


Zmieńmy nasz zestaw danych na trochę większy

In [18]:
np.random.seed(42)
df = pd.DataFrame({'val': np.random.rand(500), 'cap': np.random.randint(1,10,500)})
df

,val,cap
0,0.374540,6
1,0.950714,1
2,0.731994,9
3,0.598658,1
4,0.156019,5
...,...,...
495,0.353352,4
496,0.583656,6
497,0.077735,8
498,0.974395,1


In [19]:
problem = KnapsackProblem(df, 50)
algorithm = GA(pop_size=200,
               sampling=BinaryRandomSampling(),
               crossover=HUX(),
               mutation=BitflipMutation(),
               repair=ConsiderMaximumWeightRepair(),
               eliminate_duplicates=True)


res = minimize(problem,
               algorithm,
               termination=('n_gen', 100),
               verbose=True,
               seed=0)

n_gen  |  n_eval  |     f_avg     |     f_min    
     1 |      200 | -4.981297E+00 | -1.019492E+01
     2 |      400 | -6.231035E+00 | -1.064948E+01
     3 |      600 | -7.150589E+00 | -1.064948E+01
     4 |      800 | -8.003899E+00 | -1.079794E+01
     5 |     1000 | -8.845058E+00 | -1.198046E+01
     6 |     1200 | -9.574857E+00 | -1.198046E+01
     7 |     1400 | -1.025127E+01 | -1.311960E+01
     8 |     1600 | -1.096636E+01 | -1.383526E+01
     9 |     1800 | -1.168729E+01 | -1.463390E+01
    10 |     2000 | -1.246411E+01 | -1.539077E+01
    11 |     2200 | -1.328013E+01 | -1.634522E+01
    12 |     2400 | -1.401250E+01 | -1.640585E+01
    13 |     2600 | -1.471492E+01 | -1.850646E+01
    14 |     2800 | -1.543388E+01 | -1.899591E+01
    15 |     3000 | -1.625420E+01 | -1.899591E+01
    16 |     3200 | -1.704522E+01 | -1.984785E+01
    17 |     3400 | -1.782212E+01 | -2.245586E+01
    18 |     3600 | -1.853045E+01 | -2.245586E+01
    19 |     3800 | -1.926620E+01 | -2.245586E+01


#### Zad
Warto zaingerować w etap tworzenia inicjalnej populacji. Po pierwsze nie ma sensu wkładać do plecaka połowy dostępnych obiektów, skoro pojemności mamy tylko na kilkanaście. Po drugie zamiast wybierać je losowo warto wybrać te, które mają korzystny stosunek jakości do wagi. Zaimplentuj taką funkcję

In [20]:
from pymoo.core.sampling import Sampling

class MySampling(Sampling):

    def _do(self, problem, n_samples, **kwargs):
        X = np.full((n_samples, problem.n_var), False, dtype=bool)
        #TODO
        return X

In [21]:
algorithm = GA(pop_size=200,
               sampling=MySampling(),
               crossover=HUX(),
               mutation=BitflipMutation(),
               repair=ConsiderMaximumWeightRepair(),
               eliminate_duplicates=True)


res = minimize(problem,
               algorithm,
               termination=('n_gen', 100),
               verbose=True,
               seed=0)

n_gen  |  n_eval  |     f_avg     |     f_min    
     1 |        1 |  0.000000E+00 | -0.000000E+00
     2 |      201 | -8.646096E-01 | -2.821776E+00
     3 |      401 | -2.079479E+00 | -4.679283E+00
     4 |      601 | -3.329013E+00 | -5.849549E+00
     5 |      801 | -4.676843E+00 | -7.151387E+00
     6 |     1001 | -5.831939E+00 | -8.176994E+00
     7 |     1201 | -6.742527E+00 | -9.431336E+00
     8 |     1401 | -7.607182E+00 | -1.073984E+01
     9 |     1601 | -8.290110E+00 | -1.130620E+01
    10 |     1801 | -8.939136E+00 | -1.271432E+01
    11 |     2001 | -9.661754E+00 | -1.358227E+01
    12 |     2201 | -1.039191E+01 | -1.383659E+01
    13 |     2401 | -1.112842E+01 | -1.527238E+01
    14 |     2601 | -1.193103E+01 | -1.537905E+01
    15 |     2801 | -1.266011E+01 | -1.547080E+01
    16 |     3001 | -1.346004E+01 | -1.681096E+01
    17 |     3201 | -1.415727E+01 | -1.681096E+01
    18 |     3401 | -1.474633E+01 | -1.815292E+01
    19 |     3601 | -1.530481E+01 | -1.815292E+01


#### Zad
Przyjrzyjmy się jeszcze jak działa mutacja i krzyżowanie

In [22]:
from pymoo.core.crossover import Crossover
from pymoo.core.mutation import Mutation

class MyMutation(Mutation):
    def _do(self, problem, X, **kwargs):
        for i in range(X.shape[0]):
            #switch one bit
        return X
    

class MyCrossover(Crossover):
    def __init__(self):
        super().__init__(2, 1)

    def _do(self, problem, X, **kwargs):
        n_parents, n_matings, n_var = X.shape

        _X = np.full((self.n_offsprings, n_matings, problem.n_var), False)

        for k in range(n_matings):
            p1, p2 = X[0, k], X[1, k]
            
            #połącz obu rodziców w jedno rozwiązanie

        return _X




IndentationError: expected an indented block (280418582.py, line 8)

Przejdziemy teraz do optymalizacji wielokryterialnej. Jednym kryterium zostanie takie jakie było, drugim będzie waga plecaka

In [ ]:
class KnapsackProblemMulti(Problem):
    def __init__(self, df, C):
        super().__init__(n_var=len(df), n_obj=2, n_ieq_constr=0)
        self.df = df
        self.C = C

    def _evaluate(self, x, out, *args, **kwargs):
        f2 = (x * self.df['cap'].values).sum(1)
        f1 = np.where(f2 <= self.C, -(x * self.df['val'].values).sum(1), 1)
        out["F"] = [f1, f2] 

NSGA2 jest jednym z najpopularniejszych odmian algorytmów genetycznych do optymalizacji wielokryterialnej, nie jest jednak jedynym

In [ ]:
from pymoo.algorithms.moo.nsga2 import NSGA2

In [ ]:
problem = KnapsackProblemMulti(df, 50)
algorithm = NSGA2(pop_size=200,
               sampling=MySampling(),
               crossover=HUX(),
               mutation=BitflipMutation(),
               repair=ConsiderMaximumWeightRepair(),
               eliminate_duplicates=True)


res = minimize(problem,
               algorithm,
               termination=('n_gen', 100),
               verbose=True,
               seed=0)

In [ ]:
from pymoo.visualization.scatter import Scatter
Scatter().add(res.F).show()

In [ ]:
res.X

In [ ]:
res.F

In [ ]:
from pymoo.algorithms.moo.rnsga2 import RNSGA2

In [ ]:
algorithm = RNSGA2(ref_points=np.array([[-10, 30]]),
               pop_size=200,
               sampling=MySampling(),
               crossover=HUX(),
               mutation=BitflipMutation(),
               repair=ConsiderMaximumWeightRepair(),
               eliminate_duplicates=True)


res = minimize(problem,
               algorithm,
               termination=('n_gen', 100),
               verbose=True,
               seed=0)

In [ ]:
Scatter().add(res.F).add(np.array([[-10,30]]), label="ref_points").show()

## Zad
Rozwiąż zadanie optymalizacji wielokryterialnej stojące przed firmą kurierską. Mamy do dostarczenia przesyłki. Z jednej strony chcemy je dostarczyć jak najszybciej, z drugiej chcemy żeby to kosztowało jak najmniej. Dane wejściowe:
 - df -- zbiór zamówień z ich lokalizacją i wagą
 - t -- liczba dostępnych ciężarówek
 - p -- zbiór stacji benzynowych z lokalizacją i ceną za litr
 - fp -- funkcja określająca spalanie na 100km przy założonej wadze
 - cp -- pojemność baku
 - b -- lokalizacja bazy
 - s -- średnia prędkość w km/h
 - m -- godzinna stawka kierowcy

Na koniec wszystkie ciężarówki muszą wrócić do bazy i mieć przynajmniej 50% paliwa w baku. Wszystkie paczki muszą być dostarczone w wymaganym okresie. Pojemność ciężarówek jest nieskończona. Do obliczania dystansu użyj metryki miejskiej, współrzędne są podane w kilometrach. Ciężarówka nie musi brać całego załadunku na raz, może wracać do bazy żeby zabrać kolejne paczki

In [ ]:
np.random.seed(42)
n = 100
t = 10
df = pd.DataFrame({"latitude": np.random.randn(n)*50, 
                  "longitude": np.random.randn(n)*50, 
                  "mass": np.random.rand(n)*10})
n = 10
p = pd.DataFrame({"latitude": np.random.randn(n)*50, 
                  "longitude": np.random.randn(n)*50, 
                  "price": np.random.rand(n)*2+5})
fp = lambda m: m
cp = 150
b = np.array([0,0])
s = 40
m = 50

In [ ]:
df